<a href="https://colab.research.google.com/github/SK124/YOLOv3-on-Custom-Dataset/blob/master/custom_yolov3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


NOTE : FOR ONLY CHECKING RESULTS OR REIMPLEMENTING GO TO LOWEST PART OF NOTEBOOK

In [0]:
cp  '/content/drive/My Drive/solesensei_bdd100k.zip' /content

In [0]:
!unzip '/content/drive/My Drive/solesensei_bdd100k.zip'

FOR EASIER IMPLEMENTATION I AM USING ONLY VALIDATION FILES AS IT IS SMALL

JSON TO TEXT FORMAT 

This function will return text file containing:

location and name of jpg    file   class  x_centre  y_centre    width  height    this line continues if the image has multiple objects  

for all the images 

In [0]:
import json

categorys = ['car', 'bus', 'person', 'bike', 'truck', 'motor', 'train', 'rider', 'traffic sign', 'traffic light']


picture_width = 1282
picture_height = 720

def parseJson(jsonFile):
    '''
      INPUT
        >jsonfile in our case BDD00K 

      OUTPUT 
        > Text File with location of jpg,class,bounding box cooridnates
    '''
    objs = []
    obj = []
    info = jsonFile
    name = info['name']
    objects = info['labels']
    for i in objects:
        if(i['category'] in categorys):
            obj.append(int(i['box2d']['x1']))
            obj.append(int(i['box2d']['y1']))
            obj.append(int(i['box2d']['x2']))
            obj.append(int(i['box2d']['y2']))
            obj.append(i['category'])
            objs.append(obj)
            obj = []
    return name, objs
 
#test
file_handle = open('json2txt.txt', mode='w')
f = open("/content/bdd100k_labels_release/bdd100k/labels/bdd100k_labels_images_val.json") # Path of JSON File to be placed here
info = json.load(f)
objects = info
n = len(objects)


def bboxtrans(box_x_min, box_y_min, box_x_max, box_y_max):
    x_center = (box_x_min + box_x_max) / (2* picture_width)
    y_center = (box_y_min + box_y_max) / (2 * picture_height)
    width = (box_x_max - box_x_min) / (picture_width)
    height = (box_y_max - box_y_min) / (picture_height)
    return x_center, y_center, width, height



for i in range(n):
    an = ""
    name, result = parseJson(objects[i])
    an = "data/obj/" + name 
    for j in range(len(result)):
        cls_id = categorys.index(result[j][4])
        x, y, w, h = bboxtrans(result[j][0], result[j][1], result[j][2], result[j][3])
        an = an + ' ' +str(cls_id) + ' '+str("%.6f" %x)+' '+str("%.6f" %y)+' '+str("%.6f" %w)+' '+str("%.6f" %h)
    an = an + '\n'
    file_handle.write(an)
    #print(len(result))
    #print(an)


In [0]:
%cd /content

/content


In [0]:
mkdir textfiles

In [0]:
import re

In [0]:
regex1=re.compile('\w{8}-\w{8}')

In [0]:
regex2=re.compile('\d\s\d\.\d+\s\d\.\d+\s\d\.\d+\s\d\.\d+')

In [0]:
%cd /content/textfiles

/content/textfiles


TEXT FILE FOR EACH IMAGE

YOLOv3 requires text file with bbox co-ordinates for each image with image name.txt 

e.g bc667243-c7589903.jpg  bc667243-c7589903.txt



The following regex expression does this

In [0]:
f = open('/content/json2txt.txt', 'r')
for line in f:
  filename=re.findall(regex1,line) 
  content=re.findall(regex2,line) 
  txt_file=filename[0]+'.txt'
  with open(txt_file, 'w') as txt_file:
    for con in content:
      txt_file.write(str(con)+'\n')



f.close()


In [0]:
%cd /content/

/content


In [0]:
mkdir jpgnames

Only using few files

In [0]:
!head -n 500 json2txt.txt > train.txt

In [0]:
regex3=re.compile('\D+/\D+/\w{8}-\w{8}\.\D+')
#regex3=re.compile('\D+/\D+/\w{8}-\w{8}.jpg') will also work,for png replace with .png 

In [0]:
f = open('/content/train.txt', 'r')
with open('jpgnames.txt', 'w') as txt_file:
  for line in f:
    content=re.findall(regex3,line) 
    txt_file.write(content[0]+'\n')
        
f.close()

In [0]:
import glob
tifCounter = len(glob.glob1('/content/objecteded',"*.jpg"))
tifCounter

500

In [0]:
%cd /content
!mkdir obj

/content


In [0]:
%cd /content/bdd100k/bdd100k/images/100k/val

/content/bdd100k/bdd100k/images/100k/val


In [0]:
cp `ls | head -500` /content/obj/

In [0]:
%cd /content/textfiles/

/content/imgs


In [0]:
cp `ls | head -500` /content/obj/

In [0]:
%cd /content

/content


In [0]:
ls /content/imgs/ | wc -l

1000


In [0]:
%cd /content/

/content


In [0]:
!git clone https://github.com/AlexeyAB/darknet.git

Cloning into 'darknet'...
remote: Enumerating objects: 13057, done.
remote: Total 13057 (delta 0), reused 0 (delta 0), pack-reused 13057
Receiving objects: 100% (13057/13057), 11.77 MiB | 6.46 MiB/s, done.
Resolving deltas: 100% (8924/8924), done.


In [0]:
cp /content/obj /content/darknet/build/darknet/x64/data -r

In [0]:
%cd /content

/content


In [0]:
!tail -n 75 jpgnames.txt > test.txt

In [0]:
!head -n 425 jpgnames.txt > train1.txt

In [0]:
cp '/content/drive/My Drive/weights/darknet53.conv.74.1' /content/darknet/

In [0]:
!nvidia-smi

Fri Apr 17 08:26:04 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 440.82       Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   34C    P8    27W / 149W |      0MiB / 11441MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

In [0]:
cp '/content/drive/My Drive/BDD 500/obj' /content/darknet/data -r

In [0]:
cp /content/yolo-obj.cfg /content/darknet/

In [0]:
mv /content/darknet/data/yolo-obj.cfg /content/darknet/yolo-obj.cfg

In [0]:
rm /content/darknet/yolo-obj.cfg

In [0]:
cp /content/yolo-obj.cfg /content/darknet/

In [0]:
%cd /content/darknet/

/content/darknet


PNG TO JPG CONVERTER

There were few issues with jpg files,author recommends to use png formt hence this step,but you can try without this if it fails for you then try converting to png  

In [0]:
!apt-get install imagemagick

In [0]:
cp /content/darknet/data/labels/* /content/darknet/data/obj/

In [0]:
%cd /content/darknet/data

/content/darknet


In [0]:
!mogrify -format png *.jpg  

In [0]:
rm *'.jpg' 

Rewriting the txt file with actual location instead of relative location

In [0]:
f = open('/content/darknet/data/test.txt', 'r')

for line in f:
  line='/content/darknet/'+line
  with open('trest.txt','a') as txt_file:
    txt_file.write(line)

#f.close()


In [0]:
mv /content/darknet/trest.txt  /content/darknet/data

In [0]:
%cd /content/darknet

/content/darknet


In [0]:
mv /content/darknet/data/train.txt /content/sample_data/

In [0]:
%cd /content/darknet/data

/content/darknet/data


In [0]:
regex4=re.compile('\D+/\D+/\w{8}-\w{8}')

Renaming filenames from jpg to png 

In [0]:
f = open('/content/train.txt', 'r')
with open('train.txt', 'w') as txt_file:
  for line in f:
    content=re.findall(regex4,line) 
    txt_file.write('/content/darknet/'+content[0]+'.png'+'\n')
        
f.close()

In [0]:
%cd /content/darknet/

[Errno 2] No such file or directory: '/content/darknet/'
/content


START FROM HERE IF YOU JUST WANT TO EXPERIMENT DIRECTLY THE RESULTS

In [0]:
!git clone https://github.com/AlexeyAB/darknet

%cd darknet
!sed -i 's/OPENCV=0/OPENCV=1/' Makefile
!sed -i 's/GPU=0/GPU=1/' Makefile
!sed -i 's/CUDNN=0/CUDNN=1/' Makefile

!/usr/local/cuda/bin/nvcc --version

!make

Cloning into 'darknet'...
remote: Enumerating objects: 13221, done.
remote: Total 13221 (delta 0), reused 0 (delta 0), pack-reused 13221
Receiving objects: 100% (13221/13221), 11.91 MiB | 18.24 MiB/s, done.
Resolving deltas: 100% (9043/9043), done.
/content/darknet
nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2019 NVIDIA Corporation
Built on Sun_Jul_28_19:07:16_PDT_2019
Cuda compilation tools, release 10.1, V10.1.243
mkdir -p ./obj/
mkdir -p backup
chmod +x *.sh
g++ -std=c++11 -std=c++11 -Iinclude/ -I3rdparty/stb/include -DOPENCV `pkg-config --cflags opencv4 2> /dev/null || pkg-config --cflags opencv` -DGPU -I/usr/local/cuda/include/ -DCUDNN -Wall -Wfatal-errors -Wno-unused-result -Wno-unknown-pragmas -fPIC -Ofast -DOPENCV -DGPU -DCUDNN -I/usr/local/cudnn/include -c ./src/image_opencv.cpp -o obj/image_opencv.o
./src/image_opencv.cpp: In function ‘void draw_detections_cv_v3(void**, detection*, int, float, char**, image**, int, int)’:
./src/image_opencv.cpp:910:23: warning: v

In [0]:
cp /content/drive/'My Drive'/'BDD 500'/obj /content/ -r

In [0]:
cp /content/obj /content/darknet/data/ -r

In [0]:
cp /content/drive/'My Drive'/'BDD 500'/obj.data /content/darknet/data/

In [0]:
cp /content/drive/'My Drive'/'BDD 500'/obj.data /content/darknet/data/

In [0]:
cp /content/drive/'My Drive'/'BDD 500'/obj.names /content/darknet/data/build

In [0]:
cp /content/drive/'My Drive'/'BDD 500'/obj.names /content/darknet/data/

In [0]:
cp /content/drive/'My Drive'/'BDD 500'/obj.names /content/darknet/build/darknet/x64/data/

In [0]:
cp /content/drive/'My Drive'/'BDD 500'/obj.names /content/darknet/

In [0]:
cp /content/drive/'My Drive'/'BDD 500'/yolo-obj.cfg /content/darknet

In [0]:
cp /content/drive/'My Drive'/'BDD 500'/train.txt /content/darknet/data

In [0]:
cp /content/drive/'My Drive'/'BDD 500'/test.txt /content/darknet/data

In [0]:
cp 

USING AUTHORS WEIGHTS

In [0]:
cp '/content/drive/My Drive/weights/darknet53.conv.74.1' /content/darknet/ 

In [0]:
%cd /content/darknet/

/content/darknet


YOU CAN ASLO TRY WITH FINE TUNED WEIGHTS

In [0]:
###LAST 100 EPOCHS### cp '/content/darknet/backup/yolo-obj_last.weights' '/content/darknet/'

In [0]:
###LAST 1000 EPOCHS### cp '/content/darknet/backup/yolo-obj_1000.weights' '/content/darknet/'

In [0]:
!./darknet detector train data/obj.data yolo-obj.cfg yolo-obj_last.weights -dont_show

Streaming output truncated to the last 5000 lines.
v3 (mse loss, Normalizer: (iou: 0.75, cls: 1.00) Region 82 Avg (IOU: 0.000000, GIOU: 0.000000), Class: 0.000000, Obj: 0.000000, No Obj: 0.000063, .5R: 0.000000, .75R: 0.000000, count: 1, class_loss = 0.000149, iou_loss = 0.000000, total_loss = 0.000149 
v3 (mse loss, Normalizer: (iou: 0.75, cls: 1.00) Region 94 Avg (IOU: 0.792611, GIOU: 0.783724), Class: 0.956535, Obj: 0.379307, No Obj: 0.001749, .5R: 1.000000, .75R: 0.666667, count: 3, class_loss = 1.547460, iou_loss = 0.303380, total_loss = 1.850840 
v3 (mse loss, Normalizer: (iou: 0.75, cls: 1.00) Region 106 Avg (IOU: 0.409098, GIOU: 0.354798), Class: 0.829290, Obj: 0.369133, No Obj: 0.003071, .5R: 0.384615, .75R: 0.076923, count: 26, class_loss = 16.992233, iou_loss = 19.865295, total_loss = 36.857529 
v3 (mse loss, Normalizer: (iou: 0.75, cls: 1.00) Region 82 Avg (IOU: 0.000000, GIOU: 0.000000), Class: 0.000000, Obj: 0.000000, No Obj: 0.000011, .5R: 0.000000, .75R: 0.000000, count

In [0]:
%cd /content/darknet

/content/darknet


In [0]:
cp '/content/drive/My Drive/weights/yolo-obj_last.weights' /content/darknet

In [0]:
!./darknet detector train data/obj.data yolo-obj.cfg yolo-obj_last.weights -dont_show

 CUDA-version: 10010 (10010), cuDNN: 7.6.5, GPU count: 1  
 OpenCV version: 3.2.0
yolo-obj
 compute_capability = 370, cudnn_half = 0 
net.optimized_memory = 0 
mini_batch = 1, batch = 64, time_steps = 1, train = 1 
   layer   filters  size/strd(dil)      input                output
   0 conv     32       3 x 3/ 1    416 x 416 x   3 ->  416 x 416 x  32 0.299 BF
   1 conv     64       3 x 3/ 2    416 x 416 x  32 ->  208 x 208 x  64 1.595 BF
   2 conv     32       1 x 1/ 1    208 x 208 x  64 ->  208 x 208 x  32 0.177 BF
   3 conv     64       3 x 3/ 1    208 x 208 x  32 ->  208 x 208 x  64 1.595 BF
   4 Shortcut Layer: 1,  wt = 0, wn = 0, outputs: 208 x 208 x  64 0.003 BF
   5 conv    128       3 x 3/ 2    208 x 208 x  64 ->  104 x 104 x 128 1.595 BF
   6 conv     64       1 x 1/ 1    104 x 104 x 128 ->  104 x 104 x  64 0.177 BF
   7 conv    128       3 x 3/ 1    104 x 104 x  64 ->  104 x 104 x 128 1.595 BF
   8 Shortcut Layer: 5,  wt = 0, wn = 0, outputs: 104 x 104 x 128 0.001 BF
   9 co

In [0]:
!./darknet detector map data/obj.data yolo-obj.cfg '/content/drive/My Drive/weights/yolo-obj_1000.weights'

 CUDA-version: 10010 (10010), cuDNN: 7.6.5, GPU count: 1  
 OpenCV version: 3.2.0
 compute_capability = 370, cudnn_half = 0 
net.optimized_memory = 0 
mini_batch = 1, batch = 64, time_steps = 1, train = 0 
   layer   filters  size/strd(dil)      input                output
   0 conv     32       3 x 3/ 1    416 x 416 x   3 ->  416 x 416 x  32 0.299 BF
   1 conv     64       3 x 3/ 2    416 x 416 x  32 ->  208 x 208 x  64 1.595 BF
   2 conv     32       1 x 1/ 1    208 x 208 x  64 ->  208 x 208 x  32 0.177 BF
   3 conv     64       3 x 3/ 1    208 x 208 x  32 ->  208 x 208 x  64 1.595 BF
   4 Shortcut Layer: 1,  wt = 0, wn = 0, outputs: 208 x 208 x  64 0.003 BF
   5 conv    128       3 x 3/ 2    208 x 208 x  64 ->  104 x 104 x 128 1.595 BF
   6 conv     64       1 x 1/ 1    104 x 104 x 128 ->  104 x 104 x  64 0.177 BF
   7 conv    128       3 x 3/ 1    104 x 104 x  64 ->  104 x 104 x 128 1.595 BF
   8 Shortcut Layer: 5,  wt = 0, wn = 0, outputs: 104 x 104 x 128 0.001 BF
   9 conv     64

In [0]:
!./darknet detector map data/obj.data yolo-obj.cfg '/content/drive/My Drive/weights/yolo-obj_1000.weights'

 CUDA-version: 10010 (10010), cuDNN: 7.6.5, GPU count: 1  
 OpenCV version: 3.2.0
 compute_capability = 370, cudnn_half = 0 
net.optimized_memory = 0 
mini_batch = 1, batch = 64, time_steps = 1, train = 0 
   layer   filters  size/strd(dil)      input                output
   0 conv     32       3 x 3/ 1    416 x 416 x   3 ->  416 x 416 x  32 0.299 BF
   1 conv     64       3 x 3/ 2    416 x 416 x  32 ->  208 x 208 x  64 1.595 BF
   2 conv     32       1 x 1/ 1    208 x 208 x  64 ->  208 x 208 x  32 0.177 BF
   3 conv     64       3 x 3/ 1    208 x 208 x  32 ->  208 x 208 x  64 1.595 BF
   4 Shortcut Layer: 1,  wt = 0, wn = 0, outputs: 208 x 208 x  64 0.003 BF
   5 conv    128       3 x 3/ 2    208 x 208 x  64 ->  104 x 104 x 128 1.595 BF
   6 conv     64       1 x 1/ 1    104 x 104 x 128 ->  104 x 104 x  64 0.177 BF
   7 conv    128       3 x 3/ 1    104 x 104 x  64 ->  104 x 104 x 128 1.595 BF
   8 Shortcut Layer: 5,  wt = 0, wn = 0, outputs: 104 x 104 x 128 0.001 BF
   9 conv     64

FOR INFERENCE ONLY PUT THE PNG FILE LOCATION IN BELOW CODE 

and check the results  in 'predictions.jpg' stored in darknet director

In [0]:
!./darknet detector test data/obj.data yolo-obj.cfg yolo-obj_1000.weights <PNG FILENAME HERE>

 CUDA-version: 10010 (10010), cuDNN: 7.6.5, GPU count: 1  
 OpenCV version: 3.2.0
 compute_capability = 600, cudnn_half = 0 
net.optimized_memory = 0 
mini_batch = 1, batch = 64, time_steps = 1, train = 0 
   layer   filters  size/strd(dil)      input                output
   0 conv     32       3 x 3/ 1    416 x 416 x   3 ->  416 x 416 x  32 0.299 BF
   1 conv     64       3 x 3/ 2    416 x 416 x  32 ->  208 x 208 x  64 1.595 BF
   2 conv     32       1 x 1/ 1    208 x 208 x  64 ->  208 x 208 x  32 0.177 BF
   3 conv     64       3 x 3/ 1    208 x 208 x  32 ->  208 x 208 x  64 1.595 BF
   4 Shortcut Layer: 1,  wt = 0, wn = 0, outputs: 208 x 208 x  64 0.003 BF
   5 conv    128       3 x 3/ 2    208 x 208 x  64 ->  104 x 104 x 128 1.595 BF
   6 conv     64       1 x 1/ 1    104 x 104 x 128 ->  104 x 104 x  64 0.177 BF
   7 conv    128       3 x 3/ 1    104 x 104 x  64 ->  104 x 104 x 128 1.595 BF
   8 Shortcut Layer: 5,  wt = 0, wn = 0, outputs: 104 x 104 x 128 0.001 BF
   9 conv     64

In [0]:
!./darknet detector test data/obj.data yolo-obj.cfg yolo-obj_last.weights /content/b20841f9-cef732d5.png

 CUDA-version: 10010 (10010), cuDNN: 7.6.5, GPU count: 1  
 OpenCV version: 3.2.0
 compute_capability = 370, cudnn_half = 0 
net.optimized_memory = 0 
mini_batch = 1, batch = 64, time_steps = 1, train = 0 
   layer   filters  size/strd(dil)      input                output
   0 conv     32       3 x 3/ 1    416 x 416 x   3 ->  416 x 416 x  32 0.299 BF
   1 conv     64       3 x 3/ 2    416 x 416 x  32 ->  208 x 208 x  64 1.595 BF
   2 conv     32       1 x 1/ 1    208 x 208 x  64 ->  208 x 208 x  32 0.177 BF
   3 conv     64       3 x 3/ 1    208 x 208 x  32 ->  208 x 208 x  64 1.595 BF
   4 Shortcut Layer: 1,  wt = 0, wn = 0, outputs: 208 x 208 x  64 0.003 BF
   5 conv    128       3 x 3/ 2    208 x 208 x  64 ->  104 x 104 x 128 1.595 BF
   6 conv     64       1 x 1/ 1    104 x 104 x 128 ->  104 x 104 x  64 0.177 BF
   7 conv    128       3 x 3/ 1    104 x 104 x  64 ->  104 x 104 x 128 1.595 BF
   8 Shortcut Layer: 5,  wt = 0, wn = 0, outputs: 104 x 104 x 128 0.001 BF
   9 conv     64